In [22]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [23]:
path_allstar = os.path.join(r"C:\Users\vaibh\Documents\JobIntern\Projects\IPLTeamOfTournaments\Code\players\allstar.xlsx")
path_players = os.path.join(r"C:\Users\vaibh\Documents\JobIntern\Projects\IPLTeamOfTournaments\Code\players\iplPlayers.xlsx")
tot = pd.read_excel(path_allstar)
players = pd.read_excel(path_players)
print(tot.shape, players.shape)

(11, 17) (4095, 14)


In [24]:
players = players.iloc[:, 1:]
players.head()

,player_name,team,season,innings_played,batting_position,runs_scored,balls_faced,dismissals,fours,sixes,wickets,runs_conceded,balls_bowled
0,SC Ganguly,Kolkata Knight Riders,2007/08,6,open,168,132,5,15,8,0,0,0
1,BB McCullum,Kolkata Knight Riders,2007/08,4,open,188,92,3,13,15,0,0,0
2,RT Ponting,Kolkata Knight Riders,2007/08,4,top,39,53,4,2,1,0,0,0
3,DJ Hussey,Kolkata Knight Riders,2007/08,7,top,151,117,6,10,8,0,0,0
4,Mohammad Hafeez,Kolkata Knight Riders,2007/08,9,notRequired,41,57,4,4,1,2,68,60


In [25]:
seasons = ['2007/08', '2009', '2009/10', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
           '2018', '2019', '2020/21', '2021', '2022', '2023', '2024']
print(seasons)

['2007/08', '2009', '2009/10', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020/21', '2021', '2022', '2023', '2024']


In [26]:
tot.index = list(range(1, 12))
tot.head()

,2007/08,2009,2009/10,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020/21,2021,2022,2023,2024
1,SE Marsh,ML Hayden,SR Tendulkar,CH Gayle,CH Gayle,CH Gayle,RV Uthappa,DA Warner,V Kohli,DA Warner,KL Rahul,DA Warner,KL Rahul,KL Rahul,JC Buttler,Shubman Gill,V Kohli
2,ST Jayasuriya,AC Gilchrist,JH Kallis,SR Tendulkar,G Gambhir,MEK Hussey,DR Smith,LMP Simmons,DA Warner,S Dhawan,SR Watson,KL Rahul,S Dhawan,RD Gaikwad,KL Rahul,F du Plessis,TM Head
3,SR Watson,AB de Villiers,SK Raina,V Kohli,CL White,V Kohli,GJ Maxwell,AB de Villiers,AB de Villiers,SPD Smith,RR Pant,AB de Villiers,SS Iyer,SV Samson,HH Pandya,SA Yadav,R Parag
4,SK Raina,SK Raina,RV Uthappa,SE Marsh,SK Raina,SK Raina,SK Raina,JP Duminy,SK Raina,SK Raina,KS Williamson,SS Iyer,SA Yadav,GJ Maxwell,SV Samson,C Green,SV Samson
5,YK Pathan,RG Sharma,A Symonds,PC Valthaty,S Dhawan,SR Watson,LMP Simmons,AD Russell,CH Morris,GJ Maxwell,SP Narine,AD Russell,MP Stoinis,RA Jadeja,AD Russell,GJ Maxwell,SP Narine


In [27]:
allStarStats = tot.melt(var_name='season', value_name='player_name')
allStarStats.head()

,season,player_name
0,2007/08,SE Marsh
1,2007/08,ST Jayasuriya
2,2007/08,SR Watson
3,2007/08,SK Raina
4,2007/08,YK Pathan


In [28]:
# Add Position in Lineup
allStarStats['position'] = allStarStats.apply(
    lambda row: tot[row['season']].index[tot[row['season']] == row['player_name']].tolist()[0] 
    if row['player_name'] in tot[row['season']].values else None, axis=1
)

allStarStats.head()

,season,player_name,position
0,2007/08,SE Marsh,1
1,2007/08,ST Jayasuriya,2
2,2007/08,SR Watson,3
3,2007/08,SK Raina,4
4,2007/08,YK Pathan,5


In [32]:
def getStats(mat):
    # INSERT BATTING AVERAGE
    mat['batting_average'] = mat.apply(lambda row: row['runs_scored']/row['dismissals'] 
                                                if row['dismissals']>0 else row['runs_scored'], axis=1)
    # INSERT BATTING STRIKE RATE
    mat['batting_strike_rate'] = mat.apply(lambda row: 100*(row['runs_scored']/row['balls_faced'])
                                                if row['balls_faced']>0 else 0, axis=1)
    # INSERT BOWLING AVERAGE
    mat['bowling_average'] = mat.apply(lambda row: row['runs_conceded']/row['wickets']
                                             if row['wickets']>0 else 100, axis=1)
    # INSERT BOWLING STRIKE RATE
    mat['bowling_strike_rate'] = mat.apply(lambda row: row['balls_bowled']/row['wickets']
                                                 if row['wickets']>0 else 100, axis=1)
    # INSERT BOWLING ECONOMY RATE
    mat['economy_rate'] = mat.apply(lambda row: row['runs_conceded']/(row['balls_bowled']/6)
                                          if (row['balls_bowled']/6)>0 else 100, axis=1)
    
    return mat

def add_(mat, player_name, season):
    global allStarStats
    
    flag = ((allStarStats['season']==season) & (allStarStats['player_name']==player_name))
    # print(mat['runs_scored'].iloc[0])
    allStarStats.loc[flag, 'runs'] = mat['runs_scored'].iloc[0]
    allStarStats.loc[flag, 'batting_average'] = mat['batting_average'].iloc[0]
    allStarStats.loc[flag, 'batting_strike_rate'] = mat['batting_strike_rate'].iloc[0]
    allStarStats.loc[flag, 'fours'] = mat['fours'].iloc[0]
    allStarStats.loc[flag, 'sixes'] = mat['sixes'].iloc[0]
    allStarStats.loc[flag, 'wickets'] = mat['wickets'].iloc[0]
    allStarStats.loc[flag, 'wickets'] = mat['wickets'].iloc[0]
    allStarStats.loc[flag, 'bowling_average'] = mat['bowling_average'].iloc[0]
    allStarStats.loc[flag, 'bowling_strike_rate'] = mat['bowling_strike_rate'].iloc[0]

def insertStats(season):
    global players, tot, allStarStats
    
    lineup = list(allStarStats[allStarStats['season']==season]['player_name'])
    for i, name in enumerate(lineup):
        i += 1
        if i in [1, 2]:
            mat = players[((players['season']==season) & (players['player_name']==name)) & (players['batting_position']=='open')]
            mat = mat.drop(columns=['team', 'season', 'innings_played', 'batting_position'])
            mat = getStats(mat)
            add_(mat, name, season)
            del mat
        elif i in [3, 4]:
            mat = players[((players['season']==season) & (players['player_name']==name)) & (players['batting_position']=='top')]
            mat = mat.drop(columns=['team', 'season', 'innings_played', 'batting_position'])
            mat = getStats(mat)
            add_(mat, name, season)
            del mat
        else:
            mat = players[((players['season']==season) & (players['player_name']==name))].groupby('player_name').sum()
            mat = mat.drop(columns=['team', 'season', 'innings_played', 'batting_position'])
            mat = getStats(mat)
            add_(mat, name, season)
            del mat
            
for season in seasons:
    insertStats(season)

In [34]:
allStarStats.head()

,season,player_name,position,runs,batting_average,batting_strike_rate,fours,sixes,wickets,bowling_average,bowling_strike_rate
0,2007/08,SE Marsh,1,616.0,68.444444,139.682540,59.0,26.0,0.0,100.00,100.000
1,2007/08,ST Jayasuriya,2,514.0,42.833333,166.343042,57.0,31.0,0.0,100.00,100.000
2,2007/08,SR Watson,3,472.0,52.444444,152.258065,47.0,19.0,0.0,100.00,100.000
3,2007/08,SK Raina,4,386.0,42.888889,138.848921,33.0,15.0,0.0,100.00,100.000
4,2007/08,YK Pathan,5,435.0,31.071429,179.012346,43.0,25.0,8.0,28.75,21.125


In [35]:
# path = os.path.join(r"C:\Users\vaibh\Documents\JobIntern\Projects\IPLTeamOfTournaments\Code\players")
# path = os.path.join(path, 'tot.xlsx')
# allStarStats.to_excel(path, index=False)

In [48]:
# Rename the photo file names - 
path = os.path.join(r"C:\Users\vaibh\Documents\JobIntern\Projects\IPLTeamOfTournaments\Code\playerphotos")
files = os.listdir(path)
for file in files:
    new_name = file.replace("_", " ")
    old_path = os.path.join(path, file)
    new_path = os.path.join(path, new_name)
    os.rename(old_path, new_path)